<h1>Парсинг данных</h1>

Задача: Оценка рыночной стоимости мотоцикла с пробегом 

Сайт с данными: https://moto.drom.ru/ 

Целевая переменная: цена в рублях

Признаки: 
1. Модель мотоцикла
2. Пробег
3. Класс 
4. Год выпуска 
5. Объем двигателя 
6. Число тактов 
7. Состояние 
8. Документы
9. Город 
10. Дата публикации объявления 

Как они будут называться в таблице, разделитель ',': 

target: price

1. model
2. mileage 
3. motorcycle_class
4. year 
5. engine_capacity
6. engine_strokes 
7. damaged
8. documents
9. city
10. date 

Алгоритм:
1. Скачать все страницы (есть номер страницы, если забанят - знаю, откуда продолжить)
2. Получить из них список ссылкок (просто парсинг)
3. Потом идти по каждой ссылке (есть номер ссылки, если забанят - знаю, откуда продолжить)

<b>Импорт библиотек</b>

In [1]:
import numpy as np
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup

<b>Класс, описывающий мотоцикл</b>

In [2]:
class Motorcycle:
    def __init__(self, soup_motorcycle, index=None):
        self.soup = soup_motorcycle
        if (index is None):
            self.index = 0
        else:
            self.index = index
            
        self.warning_string_ = "Index: " + str(self.index) + "; Warning: "
        
    def warning_(self, string):
        print(self.warning_string_ + string)
        
    def parse_information(self):
        self.price = self.get_price()
        if (self.price is None):
            self.warning_('price not found')
            return False
        
        self.model = self.get_feature({"data-field" : "model"})
        if (self.model is not None):
            comma_index = self.model.find(',')
            if (comma_index != -1):
                self.model = self.model[:comma_index]
            
        self.mileage = self.get_feature({"data-field" : "motoMileage"})
        self.motorcycle_class = self.get_feature({"data-field" : "motoBodyType"})
        self.year = self.get_feature({"data-field" : "year"})
        self.engine_capacity = self.get_feature({"data-field" : "displacement"})
        self.engine_strokes = self.get_feature({"data-field" : "motoEngineType"})  
        self.damaged = self.get_feature({"data-field" : "motoDriveCondition"})
        self.documents = self.get_feature({"data-field" : "hasDocuments"})
        self.city = self.get_feature({"title" : "Выбранный город"})
        
        self.date = self.get_feature({"class" : "viewbull-header__actuality"})
        if (self.date is not None):
            comma_index = self.date.find(',')
            if (comma_index != -1):
                self.date = self.date[comma_index+2:]
            
        return True
        
    def get_price(self):
        price = self.soup.select_one('span.viewbull-summary-price__value')
        if (price is not None):
            price = price.text.strip()
            if (price[0] == '≈'):
                return None
            
        return price
    
    def get_feature(self, attrs):
        feature = self.soup.findAll(attrs=attrs)
        if (len(feature) > 0):
            feature = feature[0].text.strip()
            if (feature[-1] == '?'):
                feature = feature[:-1].strip()
        else:
            feature = None
        return feature

    def __repr__(self):
         return "Motorcycle class"
        
    def __str__(self):
        string = str(self.index) + ',' + str(self.price) + ',' + str(self.model) + ',' + \
                    str(self.mileage) + ',' + str(self.motorcycle_class) + ',' + \
                    str(self.year) + ',' + str(self.engine_capacity) + ',' + \
                    str(self.engine_strokes) + ',' + str(self.damaged) + ',' + \
                    str(self.documents) + ',' + str(self.city) + ',' + str(self.date)
        return string

<b>Информация по подключению</b>

In [3]:
cities = {'moskovskaya-obl': 171,
          'leningradskaya-obl': 65
         }
url_template = 'https://moto.drom.ru'

In [4]:
# Сайт с прокси: http://spys.one/proxys/RU/

headers = None
proxies = None
#proxies = { 'https': '5.53.19.82:56907' }

<b>Вспомогательные функции для подключения</b>

In [5]:
def get_links(url_page, proxies=None, headers=None, sleep=False, sleep_seconds=5):
    html_page = requests.get(url_page, proxies=proxies, headers=headers).content
    soup = BeautifulSoup(html_page, 'html.parser')

    if (sleep):
        time.sleep(np.random.randint(sleep_seconds))
          
    return soup.select('.bulletinLink')

In [6]:
def get_motorcycle_soup(url_motorcycle, proxies=None, headers=None, sleep=False, sleep_seconds=5):
    html_motorcycle = requests.get(url_motorcycle, proxies=proxies, headers=headers).content
    soup_motorcycle = BeautifulSoup(html_motorcycle, 'html.parser')

    if (sleep):
        time.sleep(np.random.randint(sleep_seconds))
        
    return soup_motorcycle

<b>Создание списка ссылок</b>

In [7]:
current_link = 1
with open('data/links.txt', 'a') as f_output:
    for city, last_page in cities.items():
        url_list = url_template + '/' + city + '/sale/?status=archive&page='
        for page in range(1, last_page+1):
            links = get_links(url_list + str(page), proxies, headers)

            if (links is None):
                print("Error while getting motorcycle links")
                print("Page: " + str(page))
                print("Link: " + str(current_link))
                break

            for link in links:
                print(str(current_link) + '. ' + url_template + link['href'], file=f_output)
                current_link += 1

<b>Парсинг страниц с мотоциклами</b>

In [8]:
features = "id,price,model,mileage,motorcycle_class,year,engine_capacity,engine_strokes,damaged,documents,city,date"

with open('data/links.txt', 'r') as links_file:
    with open('data/motorcycles.csv', 'a', encoding='utf-8') as motorcycle_file:
        print(features, file=motorcycle_file)
        for i, link in enumerate(links_file, 1):
            link = link[link.find('h'):]
            
            soup = get_motorcycle_soup(link, proxies, headers, False)
            if (soup is None):
                print("Error while getting motorcycle information")
                print("Link: " + str(link))
                break

            motorcycle = Motorcycle(soup, i)
            if (motorcycle.parse_information()):
                print(motorcycle, file=motorcycle_file)
            else:
                print("Error while parsing motorcycle information")
                print("Link: " + str(link))

Index: 115; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/ducati-diavel-2012-57570517.html

Index: 158; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/prodam-honda-cbr929-66695315.html

Index: 165; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/prodaju-ktm-690-duke-2012-g-v-66639732.html

Index: 177; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/harly-davidson-2013-57731456.html

Index: 197; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/bmw-k1600-gtl-66373834.html

Index: 199; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/prodam-motocikl-suzuki-desperado-vz-800-65163540.html

Index: 244; Warning: price not found
Error while pars

Index: 816; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/2015-harley-davidson-sportster-1200-62508523.html

Index: 817; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/harley-davidson-tour-glide-classic-fltc-1983-62508350.html

Index: 858; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/harley-davidson-xlh-1000-sportster-1973-62215671.html

Index: 911; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/novyj-voshod-3-61692998.html

Index: 984; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/prodam-motocikl-jawa-350-1965-54607089.html

Index: 985; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/prodam-motocikl-ural-m62-54607506.html

Index

Index: 3929; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/prodam-ural-8.103-10-46067455.html

Index: 3991; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/motocikl-boss-hoss-bhc-3-502-big-block-34344658.html

Index: 4102; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/harley-davidson-springer-softail-flstsi-45190065.html

Index: 4123; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/nonda-vfr750f-45077973.html

Index: 4146; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/suzuki-katana-44944622.html

Index: 4147; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/suzuki-44944611.html

Index: 4148; Warning: price not found
Error while parsi

Index: 5588; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/harley-davidson-electra-glide-tri-trike-lehman-kit-flhtp-36105704.html

Index: 5593; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/buell-xb12s-36956701.html

Index: 5603; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/prodazha-moto-iz-japonii-novyh-i-b-u-garantija-dostavki-podberem-36896266.html

Index: 5614; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/fryazino/sale/skuter-36868193.html

Index: 5776; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/prodam-kawasaki-vn-900-36160883.html

Index: 5838; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/prodam-honda-vt1300cx-fury-34742394.htm

Index: 7414; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/honda-nt-650v-deauvile-21903240.html

Index: 7416; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/prodam-motocikl-yamaha-yz-250-21869861.html

Index: 7439; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/motocikly-honda-gl-1995-g-v-21661531.html

Index: 7441; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/prodaju-motocykl-suzuki-gsxr-1000-2005-21610236.html

Index: 7459; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/ducati-superbike-749-dark-biposto-21557288.html

Index: 7489; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/yamaha-thunderace-1000-21362188.html

Index: 7504; 

Index: 8181; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/motocikl-bmw-r1150-rs-sporttourer-b-u-bez-probega-po-rossii-6771727.html

Index: 8201; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/perfect-condition-location-bulgaria-sofia-6914591.html

Index: 8295; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/sale/lifank-6518403.html

Index: 8309; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/prodaetsja-honda-cbr1100xx-blackbird-6425800.html

Index: 8328; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/kastom-alligator-harkov-ukraina-6207340.html

Index: 8331; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/prodam-motocikl-honda-fireblade-929-611